# Building a Task Oriented Bot

For this particular Machine Learning project I wanted to combine my interest as an aspiring Product Manager with my current work experience as a collections supervisor with the knowledge accumulated in the Machine learning module.
Building a chatbot that is able to provide answers related to invoicing and payment questions seemed for me the best way to achieve this.


### I started with putting the pseudo code together - how the interaction would look like

Pseudo code chatbot
1. User starts the conversation
2. Chatbot has answers for a list of questions that can be asked on the topic of invoicing and payment options
3. Chatbot has a slight capacity for correctly picking the right answer even when there are minor spelling mistakes
4. Topics for which chatbot can provide answers: how to pay an invoice, where to find an invoice, how to change payment method, what to do when invoice is wrong.
5. Chatbot is also able to recognize when user asks to be transferred to human agent to continue the conversation

Next steps:
1. Create the possibility for chatbot to initiate the conversation when user is logged on the page (proactivity)
2. Create the possibility for the chatbot to recognize correct invoice numbers + gather details + generate case number when extra work is needed by another dept
3. Create the possibility to have multiple interactions instead of just 1 to 1 question and answers.
4. Create the possibility of asking for feedback and registering the feedback on the helpfulness of the answer provided by the chatbot (After each question ask Was the answer helpful? If yes chatbot asks "Is there additional topics you need help with?" if yes continue with the loop. If no chatbot Thank you for reaching out to us, have a great day!. If No after Was the answer helpful? Chatbot says Sorry to hear that, let me put you in contact with an agent that can help you further)


## Code used for this project is sourced from here
https://data-flair.training/blogs/python-chatbot-project/

https://dzone.com/articles/python-chatbot-project-build-your-first-python-pro

https://www.datacamp.com/community/tutorials/building-a-chatbot-using-chatterbot

In [1]:
#pip install numpy==1.19.3

## Importing the libraries necessary for the whole project

In [2]:
import numpy as np
from tensorflow import keras #Tensorflow and Keras are used for the training of the chatbot
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import random
import nltk
from nltk.stem import WordNetLemmatizer #https://www.geeksforgeeks.org/python-lemmatization-with-nltk/#:~:text=Lemmatization%20is%20the%20process%20of,similar%20meaning%20to%20one%20word. Lemmatization is the process of grouping together the different inflected forms of a word so they can be analysed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meaning to one word.
lemmatizer = WordNetLemmatizer()
import json
import pickle #https://docs.python.org/3/library/pickle.html The pickle module implements binary protocols for serializing and de-serializing a Python object structure. “Pickling” is the process whereby a Python object hierarchy is converted into a byte stream, and “unpickling” is the inverse operation, whereby a byte stream (from a binary file or bytes-like object) is converted back into an object hierarchy. Pickling (and unpickling) is alternatively known as “serialization”, “marshalling,” 1 or “flattening”; however, to avoid confusion, the terms used here are “pickling” and “unpickling”.
import nltk
nltk.download('punkt') #https://www.kite.com/python/docs/nltk.punkt Punkt Sentence Tokenizer This tokenizer divides a text into a list of sentences, by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences. It must be trained on a large collection of plaintext in the target language before it can be used. The NLTK data package includes a pre-trained Punkt tokenizer for English.
import nltk
nltk.download('wordnet') #https://www.tutorialspoint.com/how-to-get-synonyms-antonyms-from-nltk-wordnet-in-python#:~:text=The%20WordNet%20is%20a%20part,then%20download%20the%20WordNet%20package. The WordNet is a part of Python's Natural Language Toolkit. It is a large word database of English Nouns, Adjectives, Adverbs and Verbs. These are grouped into some set of cognitive synonyms, which are called synsets.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Merti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Merti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Importing the "intents" file in which we create the potential questions from the user and potential answers from the chatbot, where we have the following structure

#tag - type of topic discussed 
#patterns - potential user input
#responses - variation of possible chatbot responses which can be given based on the patterns

In [3]:
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

In this first phase the potential questions from user (the patterns) are split, tokenized, linked to a tag (so there is a connection between the tag-topic and the pattern - user question). by printing documents we see this connection made between the tag and the pattern

In [4]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)        
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Hay'], 'greeting'), (['Good', 'morning'], 'greeting'), (['Good', 'day'], 'greeting'), (['Good', 'evening'], 'greeting'), (['Good', 'afternoon'], 'greeting'), (['Hiya'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Have', 'a', 'nice', 'day'], 'goodbye'), (['Have', 'a', 'great', 'day'], 'goodbye'), (['See', 'ya'], 'goodbye'), (['Later'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thanks', 'for', 'the', 'help'], 'thanks'), (['Who', 'are', 'you', '?'], 'name'), (['What', 'are', 'you', '?'], 'name'), (['Who', 'am', 'I', 'talking', 'to', '?'], 'name'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['what', 'are', 'you', 'called'], 'name'), (['what', 'are', 'they', 'c

2 files words and classes are created based on the information from the lemmatization. the tags are turned into 13 classes. there are 112 unique lemmatized words based on the information

In [5]:
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

214 documents
16 classes ['changepayment', 'findinvoice', 'goodbye', 'greeting', 'help', 'human', 'inputinvoiceamount', 'inputinvoicedetails', 'inputinvoiceduedate', 'name', 'payinvoice', 'thanks', 'wronginvoice', 'wronginvoiceamount', 'wronginvoicedetails', 'wronginvoiceduedate']
258 unique lemmatized words ["'s", "'ve", '01.10.2020', '04.04', '1', '10', '100', '10000', '10118', '11', '12', '123', '15.06', '150', '1500', '1600', '17.08', '18th', '19th', '2', '200', '21st', '221b', '23', '25.12.2020', '28', '300', '345', '350', '3rd', '4059', '56', '5th', '75', '90068', '98', 'a', 'access', 'actual', 'address', 'advice', 'afternoon', 'agent', 'am', 'amend', 'amount', 'amsterdam', 'an', 'anymore', 'anyone', 'are', 'aud', 'available', 'avenue', 'baker', 'be', 'before', 'being', 'belong', 'bill', 'billing', 'buckingham', 'but', 'bye', 'ca', 'cad', 'call', 'called', 'calling', 'can', 'card', 'cash', 'cc', 'champ', 'change', 'charge', 'charged', 'chf', 'cny', 'contact', 'copy', 'correct', '

In this step the training data is created with the bag of words method. 

In [6]:
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


C:\Users\Merti\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Create the neural network model using the Sequential method. Sequential API is the easiest model to build and run in Keras. A sequential model allows us to create models layer by layer in a step by step fashion.

In [7]:
# deep neural network model
model = Sequential()
model.add(Dense(200, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model is created")

Epoch 1/200
43/43 [==============================] - 0s 973us/step - loss: 2.7317 - accuracy: 0.0888
Epoch 2/200
43/43 [==============================] - 0s 953us/step - loss: 2.4568 - accuracy: 0.2383
Epoch 3/200
43/43 [==============================] - 0s 1000us/step - loss: 2.0884 - accuracy: 0.3738
Epoch 4/200
43/43 [==============================] - 0s 1ms/step - loss: 1.7500 - accuracy: 0.4252
Epoch 5/200
43/43 [==============================] - 0s 999us/step - loss: 1.4557 - accuracy: 0.5374
Epoch 6/200
43/43 [==============================] - 0s 1000us/step - loss: 1.3007 - accuracy: 0.5981
Epoch 7/200
43/43 [==============================] - 0s 1ms/step - loss: 1.1423 - accuracy: 0.6402
Epoch 8/200
43/43 [==============================] - 0s 953us/step - loss: 1.0431 - accuracy: 0.6402
Epoch 9/200
43/43 [==============================] - 0s 1ms/step - loss: 0.8308 - accuracy: 0.7617
Epoch 10/200
43/43 [==============================] - 0s 1ms/step - loss: 0.7767 - accuracy: 0.

43/43 [==============================] - 0s 860us/step - loss: 0.0468 - accuracy: 0.9860
Epoch 83/200
43/43 [==============================] - 0s 837us/step - loss: 0.0286 - accuracy: 0.9907
Epoch 84/200
43/43 [==============================] - 0s 790us/step - loss: 0.0256 - accuracy: 0.9953
Epoch 85/200
43/43 [==============================] - 0s 860us/step - loss: 0.0186 - accuracy: 0.9953
Epoch 86/200
43/43 [==============================] - 0s 814us/step - loss: 0.0267 - accuracy: 0.9907
Epoch 87/200
43/43 [==============================] - 0s 813us/step - loss: 0.0122 - accuracy: 1.0000
Epoch 88/200
43/43 [==============================] - 0s 860us/step - loss: 0.0187 - accuracy: 0.9907
Epoch 89/200
43/43 [==============================] - 0s 860us/step - loss: 0.0181 - accuracy: 1.0000
Epoch 90/200
43/43 [==============================] - 0s 813us/step - loss: 0.0204 - accuracy: 1.0000
Epoch 91/200
43/43 [==============================] - 0s 837us/step - loss: 0.0264 - accuracy: 

43/43 [==============================] - 0s 907us/step - loss: 0.0040 - accuracy: 1.0000
Epoch 162/200
43/43 [==============================] - 0s 906us/step - loss: 0.0055 - accuracy: 1.0000
Epoch 163/200
43/43 [==============================] - 0s 860us/step - loss: 0.0117 - accuracy: 1.0000
Epoch 164/200
43/43 [==============================] - 0s 813us/step - loss: 0.0050 - accuracy: 1.0000
Epoch 165/200
43/43 [==============================] - 0s 813us/step - loss: 0.0070 - accuracy: 1.0000
Epoch 166/200
43/43 [==============================] - 0s 860us/step - loss: 0.0098 - accuracy: 1.0000
Epoch 167/200
43/43 [==============================] - 0s 837us/step - loss: 0.0054 - accuracy: 1.0000
Epoch 168/200
43/43 [==============================] - 0s 906us/step - loss: 0.0068 - accuracy: 1.0000
Epoch 169/200
43/43 [==============================] - 0s 837us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 170/200
43/43 [==============================] - 0s 837us/step - loss: 0.0038 - a

we can also check the model summary to see how many parameters were trained

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               51800     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1616      
Total params: 73,516
Trainable params: 73,516
Non-trainable params: 0
_________________________________________________________________


In [9]:
#pip install tf-nightly

In this step we load the model trained and created previously, as well as the intents file, the words and classes file

In [10]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

Here we create the functions pipeline for the chatbot activity:
    1. sentences inputted by the user are cleaned up, tokenized, reduced to base form
    2. cleaned sentences are turned into bag of words
    3. based on the bag of words a function is predicting in which of the 13 classes the user sentence needs to be placed
    4. another function is creating a randomization of retrieving one of the response answers based on the tags
    5. last function is making the connection between the user input and the answer the chatbot retrieves

In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

This last step is simply to create the interface for interaction with the chatbot in a nice environment

In [12]:
#Creating tkinter GUI
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        ChatBox.insert(END, "Fin: " + res + '\n\n')           
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)

root = Tk()
root.title("Fin the Finance Assistance Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="60", font="Arial",)
ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window

scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="double_arrow")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message

SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

#Create the box to enter message

EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")

#EntryBox.bind("<Return>", send)

#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
root.mainloop()

In [13]:
#python train_chatbot.py

In [14]:
#python chatgui.py

### The final step is to test the accuracy of the answers given by the chatbot. 

#### With a score of 9 correct answers from 12 questions (Sample as below), we can consider this a fairly reliable chatbot for the task. We need to further test it with additional users and additional scenarios

You: hei good evening

Correct Fin: Hi, I am Fin, your Finance assistant chatbot, how can I help you today?

You: lets see if you can help as my invoice is incorrect

Correct Fin: I am sorry you consider your invoice to be incorrect. Can you let me know which of these areas is incorrect:Wrong amount/Wrong Billing Details/Wrong Due Date)

You: the amount on the invoice is wrong

Correct Fin: It seems you think the amount of the invoice is wrong. Please mention the invoice number, amount you think should have been charged and the reason why and I will open a case for our Finance Department to come back to you in the shortest time)

You: ok this is great thank you have a nice day

Correct Fin: Have a nice day! Before you go, do not forget to check the details for the recommended automated payment methods (link)

You: there is something else. i need to know what are my payment options

Wrong Fin: The available payment methods are credit card, bank transfer and automated payment. You can change your payment method here ...

You: perfect this helps

Wrong Fin: Yes please tell me your problem so I can assist you

You: i do not need any other support. thank you very much

Wrong Fin: Yes please tell me your problem so I can assist you

You: thank you! have a great day

Correct Fin: Have a nice day! Before you go, do not forget to check the details for the recommended automated payment methods (link)

You: where can i retrieve a copy of my invoice

Correct Fin: You can find a copy of your invoice by accessing this path ...

You: and how can i pay this invoice

Correct Fin: The available payment methods are credit card, bank transfer and automated payment. You can process payments from here ...

You: aha and what if i want to change the payment method?

Correct Fin: The available payment methods are credit card, bank transfer and automated payment. You can change your payment method here ...

You: this is helpful thanks

Correct Fin: Great to hear I could be of help! Before you go, do not forget to check the details for the recommended automated payment methods (link)

#### Another scenario tested is when certain words are mispelled, in 6 out of 8 tests even with typos the chatbot still provides the right answer

You: how can i pay my otstanding invoice

Correct Fin: The available payment methods are credit card, bank transfer and automated payment. You can process payments from here ...

You: i need a cpy of my invoice

Correct Fin: You can find a copy of your invoice by accessing this path ...

You: my invice is wrong

Correct Fin: I am sorry you consider your invoice to be incorrect. Can you let me know which of these areas is incorrect:Wrong amount/Wrong Billing Details/Wrong Due Date)

You: payment mthod to be changed

Correct Fin: The available payment methods are credit card, bank transfer and automated payment. You can change your payment method here ...

You: please get me a hman agent

Correct Fin: Sorry to hear I cannot help you, let me connect you with a human agent. Have a nice day!

You: amont is wrng

Wrong Fin: Hello, I am Fin, your Finance assistant chatbot, how can I help you today?

You: billng detals is incorrect

Wrong Fin: It seems you think the amount of the invoice is wrong. Please mention the invoice number, amount you think should have been charged and the reason why and I will open a case for our Finance Department to come back to you in the shortest time)


# Next steps

In order to improve accuracy, further testing is needed to be done by additional users to account for cultural differences and different types of approaches when asking questions, and the input to be reintegrated in the intents file.

Next step is also to create more complexity in the discussion with the chatbot, allowing for more than just 1 to 1 question and answers, and allowing the chatbot to follow up or ask additional clarifying questions.

Another potential improvement is to allow the chatbot to initiate conversation when the user is on the page, which increases the proactivity and user experience.

And another important improvement would be to ask for feedback on the answer to each question and quantify the feedback in order to improve the algorithm.

# Conclusion

By using Keras on TensorFlow and NLTK libraries, having a detailed intents file used for training the model, having a low loss and high accuracy after the training, has allowed for the creation of a reliable task based chatbot that can correctly answer with high confidence on a restricted number of preset topics.

For the purpose of this task based chatbot using Keras and NLTK is a good choice. For the future an idea would be to explore other methods and compare accuracy in order to understand if chatbot can be improved.